# Проведем анализ с помощью SQL

**Цель проекта:** провести анализ продаж розничной сети с помощью SQL (SQLalchemy)

In [1]:
!pip install sqlalchemy

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
features = pd.read_csv('Features data set.csv')
sales = pd.read_csv('sales data-set.csv')
stores = pd.read_csv('stores data-set.csv')

In [4]:
# создаем engine для SQLite (база будет храниться в файле 'retail.db')
engine = create_engine('sqlite:///retail.db')

# Сохраняем датафреймы в базу
stores.to_sql('stores', con=engine, index=False, if_exists='replace')
features.to_sql('features', con=engine, index=False, if_exists='replace')
sales.to_sql('sales', con=engine, index=False, if_exists='replace')

421570

## Количество уникальных магазинов в таблице sales

In [5]:
query = '''
SELECT COUNT(DISTINCT Store) AS unique_stores_cnt
FROM stores
'''
pd.read_sql(query, con=engine)

,unique_stores_cnt
0,45


## Средняя сумма продаж по каждому магазину

In [6]:
query = """
SELECT
    Store,
    ROUND(AVG(Weekly_Sales), 2) AS avg_weekly_sales
FROM sales
GROUP BY Store
ORDER BY avg_weekly_sales DESC
LIMIT 5
"""
pd.read_sql(query, con=engine)

,Store,avg_weekly_sales
0,20,29508.30
1,4,29161.21
2,14,28784.85
3,13,27355.14
4,2,26898.07


## Средние продажи по типам магазинов

In [7]:
query = """
SELECT
    s.Type,
    AVG(sa.Weekly_Sales) AS avg_sales
FROM sales sa
JOIN stores s
ON sa.Store = s.Store
GROUP BY s.Type
ORDER BY avg_sales DESC
"""
pd.read_sql(query, con=engine)

,Type,avg_sales
0,A,20099.568043
1,B,12237.075977
2,C,9519.532538


## Средние продажи в разных типах магазинов по праздничным и обычным неделям

In [8]:
query = """
SELECT
    s.Type,
    CASE WHEN sa.IsHoliday = 1 THEN 'Праздник' ELSE 'Обычный' END AS IsHoliday,
    ROUND(AVG(Weekly_Sales), 2) AS avg_sales
FROM sales sa
JOIN stores s
ON sa.Store = s.Store
GROUP BY s.Type, sa.IsHoliday
ORDER BY s.Type, sa.IsHoliday
"""
pd.read_sql(query, con=engine)

,Type,IsHoliday,avg_sales
0,A,Обычный,20008.75
1,A,Праздник,21297.52
2,B,Обычный,12153.07
3,B,Праздник,13346.16
4,C,Обычный,9518.53
5,C,Праздник,9532.96


## Топ-5 департаментов по суммарным продажам

In [9]:
query = """
SELECT
    Dept,
    SUM(Weekly_Sales) AS total_sales
FROM sales
GROUP BY Dept
ORDER BY total_sales DESC
LIMIT 5
"""
pd.read_sql(query, con=engine)

,Dept,total_sales
0,92,4.839433e+08
1,95,4.493202e+08
2,38,3.931181e+08
3,72,3.057252e+08
4,90,2.910685e+08


## Магазины, у которых средние продажи выше среднего по всем магазинам

In [10]:
query = """
SELECT
    Store,
    AVG(Weekly_Sales) AS avg_store_sales
FROM sales
GROUP BY Store
HAVING AVG(Weekly_Sales) > (SELECT AVG(Weekly_Sales) FROM sales)
ORDER BY avg_store_sales DESC
"""
pd.read_sql(query, con=engine)

,Store,avg_store_sales
0,20,29508.301592
1,4,29161.210415
2,14,28784.851727
3,13,27355.136891
4,2,26898.070031
5,10,26332.303819
6,27,24826.984536
7,6,21913.243624
8,1,21710.543621
9,39,21000.763562


## Сравнение динамики продаж для каждого департамента каждого магазина по неделям

In [11]:
query = """
SELECT
    Store,
    Dept,
    Date,
    Weekly_Sales,
    LAG(Weekly_Sales) OVER (PARTITION BY Store, Dept ORDER BY Date) AS prev_week_sales,
    Weekly_Sales - LAG(Weekly_Sales) OVER (PARTITION BY Store, Dept ORDER BY Date) AS diff_sales_abs
FROM sales
ORDER BY Store, Dept, Date
"""
pd.read_sql(query, con=engine)

,Store,Dept,Date,Weekly_Sales,prev_week_sales,diff_sales_abs
0,1,1,01/04/2011,20398.09,NaN,NaN
1,1,1,01/06/2012,16065.49,20398.09,-4332.60
2,1,1,01/07/2011,15363.50,16065.49,-701.99
3,1,1,01/10/2010,20094.19,15363.50,4730.69
4,1,1,02/03/2012,20113.03,20094.19,18.84
...,...,...,...,...,...,...
421565,45,98,30/07/2010,5.50,763.60,-758.10
421566,45,98,30/09/2011,727.05,5.50,721.55
421567,45,98,30/12/2011,553.21,727.05,-173.84
421568,45,98,31/08/2012,346.04,553.21,-207.17


## Для каждого магазина - неделя с наибольшими продажами

In [12]:
query = """
WITH store_max_weeks AS (
    SELECT
        Store,
        Date,
        Weekly_Sales,
        ROW_NUMBER() OVER (PARTITION BY Store ORDER BY Weekly_Sales DESC) AS rn
    FROM sales
)
SELECT
    Store,
    Date,
    Weekly_Sales
FROM store_max_weeks
WHERE rn = 1
ORDER BY Weekly_Sales DESC
"""
pd.read_sql(query, con=engine)

,Store,Date,Weekly_Sales
0,10,26/11/2010,693099.36
1,35,25/11/2011,649770.18
2,14,26/11/2010,474330.10
3,20,26/11/2010,422306.25
4,27,26/11/2010,420586.57
5,22,26/11/2010,393705.20
6,4,25/11/2011,385051.04
7,23,26/11/2010,369830.98
8,12,25/11/2011,360140.66
9,28,26/11/2010,355356.39
